<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2022-1/blob/main/7_Un_poco_de_todo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En esta sesión veremos algunos ejemplos de problemas con la finalidad de reforzar lo visto en esta unidad.

**Ejemplo 1**. Supongamos que se tiene una función $f$ integrable en el intervalo $[a,b]$ y deseamos calcular el valor de $\int_{a}^{b} f$. 

¿Cómo podemos aprovechar ideas probabilísticas para atacar este problema? Podemos seleccionar de manera aleatoria $n$ reales en el intervalo $[a,b]$, para una $n$ lo suficientemente grande, el promedio de los valores de $f$ es dichos puntos multiplicado por la longitud del intervalo aproximará la integral deseada. Veamos un ejemplo de esto, para integrar la función seno en el intervalo $[0,\pi]$, en donde probaremos con diferentes valores de $n$.



In [ ]:
import numpy as np
import random
import math

def integral(a, b, n):
  tot = 0
  for i in range(0,n):
    x = random.uniform(a,3.14)
    # print(x)
    tot += math.sin(x)
  return tot*(b-a)/n

print(integral(0,np.pi,10))
print(integral(0,np.pi,100))
print(integral(0,np.pi,1000))
print(integral(0,np.pi,10000))
print(integral(0,np.pi,100000))
print(integral(0,np.pi,1000000))

2.1541890359223075
1.883883984931964
1.923788058439326
2.0004026571579683
2.002561467392846
2.0001477149204923


**Ejemplo 2.** Se tiene una lista $L$ con $n$ enteros no necesariamente distintos. Para cualquier lista, se define su *diversidad* como el total de elementos distintos que tiene la lista en cuestión. Si a cada elemento de $L$ se le puede sumar $1$ o dejarlo como está, determina un algoritmo que nos permita conocer la mayor diversidad que se puede alcanzar al hacer estos cambios en la lista.

Una forma en la que podemos proceder es checando todas las posibilidades, es decir, si cada elemento lo cambiamos o lo dejamos como está. Sin embargo esto es exponencial, nos tomaría al menos $O(2^n)$, ¿se puede mejorar esto?

Consideremos el siguiente algoritmo greedy. Recorremos la lista de izquierda a derecha, y vamos agregando los elementos a un conjunto, en cada paso verificamos si el elemento en el que estamos ya pertenece al conjunto, de ser así le sumamos $1$ y lo agregamos. Este algoritmo tiene complejidad en tiempo $O(n)$, (en otros lenguajes de programación puede tomar $O(n logn)$ por el uso de un conjunto). ¿Es correcto este algoritmo? ¿Qué pasa con las listas `[2,2,3]` y `[3,2,2]`?

Podemos modificar un poco el algoritmo anterior, ordenando primero nuestra lista, y procediendo con el algoritmo greedy mencionado. Veamos una implementación de este algoritmo.

In [ ]:
def max_div(L):
  L.sort()
  S = {L[0]}
  for i in range(1, len(L)):
    if(L[i] in S):
      L[i] += 1
    S.add(L[i])
  return len(S)

print(max_div([1, 2, 2, 2, 5, 6]))
print(max_div([3,2,2]))
print(max_div([2,2,3,3,4,4]))
print(max_div([1, 1, 3, 4, 4, 5]))


5
3
4
6


Como hemos visto anteriormente, probar que un algoritmo greedy es correcto suele ser complicado, por lo que hay ocasiones en las que uno puede preferir otro tipo de algoritmo, para ir más a la segura. ¿Podemos resolver este problema usando recursión o programación dinámica? Consideremos la lista ordenada de menor a mayor. Recorremos la lista de izquierda a derecha, hasta encontrar un índice $i$ tal que $L[i] + 1 < L[i+1]$. Tenemos entonces que los primeros $i+1$ elementos cubren a cada entero del intervalo $[L[0], L[i]]$ al menos una vez, y al hacer cambios podremos cubrir a lo más un elemento más, lo que es posible solo si existe elemento repetido en dicho intervalo. Con esto en mente podemos implementar un algoritmo diferente al greedy.



In [ ]:
def max_div2(L):
  L.sort()
  ans = 0
  prev = 0 # guarda el índice de hasta la izquierda de cada intervalo contiguo
  rep = False
  for i in range(0, len(L) - 1):
    if(L[i] + 1 < L[i+1]):
      if(rep): 
        ans += L[i] - L[prev] + 2
      else:
        ans += L[i] - L[prev] + 1
      prev = i + 1
      rep = False
    else:
      if(i > 0 and L[i] == L[i-1]):
        rep = True
  return ans

print(max_div([1, 2, 2, 2, 5, 6]))
print(max_div([3,2,2]))
print(max_div([2,2,3,3,4,4]))
print(max_div([1, 1, 3, 4, 4, 5]))

5
3
4
6


**Ejemplo 3.** Se tiene un tablero de $n \times 2$, y en cada casilla hay un entero entre $0$ y $1000$ escrito en ella. Se quieren 'eliminar' $k$ de estas casillas de modo que la suma de las casillas removidas sea la menor posible, y que se pueda llegar de un extremo del tablero al otro, caminando por casillas que son adyacentes por lado. 

¿Cómo proceder con este problema? Una primera observación importante es notar que si hay $m$ columnas consecutivas tales que en cada una de ellas se remueve alguna casilla (notemos que en cada columna se elimina a lo más una de sus casillas), entonces todas las casillas que se remueven de esas $m$ columnas pertenecen a la misma fila. 

Esta observación nos va a permitir crear una recursión en términos de la longitud del tablero y la cantidad de casillas por eliminar, si $f(l,e)$ nos dice la respuesta al problema considerando las primeras $l$ columnas, eliminando $e$ casillas en ese subtablero, se tiene la recursión:

$$f(l,e) = min_{i = 0}^{e}\{(f(l-1-i, e-i)) + min(S1(l-i, l), S2(l-i,l))\}$$

Donde $S1(a,b), S2(a,b)$ son las sumas de los elementos de la primer y segunda fila comprendidos en las columnas $a, a+1, \dots, b$, respectivamente.

Con esto en mente podemos implementar el algoritmo.

In [ ]:
tablero = [[3,2,1,1,3,0],[1,1,2,3,3,0]]
# tablero = [[7,4,3,5,7,10,0,3,6,7],[8,9,7,9,2,3,10,2,3,9]]
n = len(tablero[0])
k = 5

acum = [[0],[0]]
for i in range(0, n): # Esto nos permitirá calcular las sumas S1, S2 de manera rápida
  acum[0].append(tablero[0][i] + acum[0][-1])
  acum[1].append(tablero[1][i] + acum[1][-1])

S = acum[0][-1] + acum[1][-1]

dp =  [[-1 for x in range(k+1)] for y in range(n+1)]

def quitar_casillas(l,e):
  if(dp[l][e] != -1):
    return dp[l][e]
  if(e == 0):
    return 0
  if(e == l):
    dp[l][e] = min(acum[0][l] - acum[0][0], acum[1][l] - acum[0][0])
  else:
    mini = 2000*n + 2 # Ponemos un 'infinito'
    for i in range(0, e+1):
      aux = quitar_casillas(l-1-i, e-i) + min(acum[0][l] - acum[0][l-i], acum[1][l] - acum[1][l-i])
      mini = min(mini, aux)
      # print(aux, i, mini)
    dp[l][e] = mini
  # print(l,e, dp[l][e])
  return dp[l][e]

print(quitar_casillas(n, k))
print(S)


6
20


**Ejercicios.**

1.   Describe otro algoritmo probabilístico que se pueda utilizar para aproximar la integral de cierta función. Implementa tu algoritmo para aproximar la integral de la función $log(x)$ en el intervalo $[1,2]$.
2.   Decimos que un entero es súper divisible si es múltiplio de cada uno de sus dígitos distintos de $0$. Por ejemplo, $102$ y $132$ son súper divisibles, pero $282$ y $119$ no lo son. Describe un algoritmo tal que dado un entero positivo $n$, encuentre el menor entero $m$ tal que $n < m$ y $m$ es súper divisible. Implementa tu algoritmo (comprueba los casos $n= 14, 28, 119, 282, 102, 132, 209$) y analiza su complejidad en tiempo.
(Hint: ¿Qué pasa si un número es múltiplo de $mcd(1,2,\dots,9)$)?



*Ejercicio 1*. Describe a continuación el algoritmo solicitado.

In [ ]:
# Aquí va la implementación de tu algoritmo

*Ejercicio 2.* Describe aquí tu algoritmo y analiza su complejidad en tiempo.

In [ ]:
# Aquí va la implementación de tu algoritmo, recuerda comprobarlo para los casos n = 14, 28, 119, 282, 102, 132, 209